# Improving Neural Net Performance
**Suggested time to spend on exercise**: 20 minutes

In this exercise, we will focus on improving the performance of the NN we trained in the previous exercise. Normalizing the features is particularly important to obtaining good performance. Another way is by trying different optimization algorithms. Note that neither of these methods are specific to neural networks - they are effective means to improve most types of models.

First, we'll load the data.

In [ ]:
#@test {"output": "ignore"}

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.google as tf
from IPython import display
from google3.pyglib import gfile
from sklearn import metrics


def preprocess_features(california_housing_dataframe):
  """This function takes an input dataframe and returns a version of it that has
  various features selected and pre-processed.  The input dataframe is expected
  to contain data from the california_housing data set."""
  processed_features = california_housing_dataframe[
    ["latitude",
     "longitude",
     "housingMedianAge",
     "totalRooms",
     "totalBedrooms",
     "population",
     "households",
     "medianIncome"]].copy()
  processed_features["roomsPerPerson"] = (
    california_housing_dataframe["totalRooms"] /
    california_housing_dataframe["population"])
  # Feel free to add other synthetic features here.
  return processed_features


def preprocess_targets(california_housing_dataframe):
  """This function selects and potentially transforms the output target from
  a dataframe containing data from the california_housing data set.  Object
  returned is a pandas Series."""
  processed_targets = pd.DataFrame()
    # Scale the target to be in units of thousands of dollars.
  processed_targets["medianHouseValue"] = (
    california_housing_dataframe["medianHouseValue"] / 1000.0)
  return processed_targets


# Load in the raw data.  Note that there's a separate test data set that we
# will leave untouched for now.
raw_training_df = pd.read_csv(
  gfile.Open("/placer/prod/home/ami/mlcc/california_housing/v1/train.csv"),
  sep=",")
# Randomize the data before selecting train / validation splits.
raw_training_df = raw_training_df.reindex(
  np.random.permutation(raw_training_df.index))

# Choose the first 12000 (out of 17000) examples for training.
training_examples = preprocess_features(raw_training_df.head(12000))
training_targets = preprocess_targets(raw_training_df.head(12000))

# Choose the last 5000 (out of 17000) examples for validation.
validation_examples = preprocess_features(raw_training_df.tail(5000))
validation_targets = preprocess_targets(raw_training_df.tail(5000))

Next, we train our NN.

In [ ]:
#@test {"output": "ignore", "timeout": 180}

LEARNING_RATE = 0.0007  # @param
STEPS = 5000  # @param
BATCH_SIZE = 70  # @param
HIDDEN_UNITS = [10, 10]  # @param
periods = 10
steps_per_period = STEPS / periods

# Set up our NN with the desired learning settings.
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(
  training_examples)
dnn_regressor = tf.contrib.learn.DNNRegressor(
  feature_columns=feature_columns,
  hidden_units=HIDDEN_UNITS,
  optimizer=tf.GradientDescentOptimizer(learning_rate=LEARNING_RATE),
  gradient_clip_norm=5.0
)

print "Training model..."
print "RMSE:"
root_mean_squared_errors_training = []
root_mean_squared_errors_validation = []
for period in range (0, periods):
  dnn_regressor.fit(
    training_examples,
    training_targets,
    steps=steps_per_period,
    batch_size=BATCH_SIZE
  )
  predictions_validation = dnn_regressor.predict(validation_examples)
  predictions_training = dnn_regressor.predict(training_examples)

  root_mean_squared_error_validation = math.sqrt(metrics.mean_squared_error(
    predictions_validation, validation_targets))
  root_mean_squared_error_training = math.sqrt(metrics.mean_squared_error(
    predictions_training, training_targets))

  root_mean_squared_errors_validation.append(root_mean_squared_error_validation)
  root_mean_squared_errors_training.append(root_mean_squared_error_training)

  print "  period %02d : %3.2f" % (period, root_mean_squared_error_training)

# Output a graph of loss metrics over periods.
plt.ylabel("RMSE")
plt.xlabel("Periods")
plt.title("Root Mean Squared Error vs. Periods")
plt.plot(root_mean_squared_errors_training, label='training')
plt.plot(root_mean_squared_errors_validation, label='validation')
plt.legend()

# Display some summary information.
print "Final RMSE (on training data):   %0.2f" % root_mean_squared_error_training
print "Final RMSE (on validation data): %0.2f" % root_mean_squared_error_validation

### Task 1: Normalize the features using linear scaling.

**Normalize the inputs to the scale -1, 1.  You can use the helper method in the following cell.**

**Spend about 5 minutes training and evaluating on the newly normalized data.  How well can you do?**

As a rule of thumb, NN's train best when the input features are roughly on the same scale.

It can be a good standard practice to normalize the inputs to fall within the range -1, 1. This helps SGD not get stuck taking steps that are too large in one dimension, or too small in another. Fans of numerical optimization may note that there's a connection to the idea of using a preconditioner here.

Sanity check your normalized data.  (What would happen if you forgot to normalize one feature?)
